<a href="https://colab.research.google.com/github/Kussil/Financial_Sentiment_LLM/blob/main/03_Sentiment_Analysis/LLama/LLama_Colab_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPU info

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Jul 22 20:21:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   43C    P8              12W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Connecting to G-drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Installing and importing libraries

In [3]:
# Install necessary libraries
!pip install -q -U langchain langchain_community transformers bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.0/990.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 757.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 69.3 MB/s eta 0:00:00


In [4]:
import os
import json
import pandas as pd
import torch
from tqdm import tqdm
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import HfFolder, HfApi
from google.colab import userdata
import re
import time

# Hugging Face Using Secret Token

In [5]:
# Retrieve the token from Colab secrets
hf_token = userdata.get('HF_TOKEN')
if hf_token:
    HfFolder.save_token(hf_token)
    api = HfApi()
    user_info = api.whoami()
    if user_info:
        print("Connection to Hugging Face was successful.")
    else:
        print("Failed to connect to Hugging Face. Please check your token.")
else:
    print("Hugging Face token not found. Please set the HF_TOKEN environment variable.")


Connection to Hugging Face was successful.


# Connecting to GitHub Repo

In [6]:
# Import GitHub token with Google secrets and clone the repository
GITHUB_TOKEN = userdata.get('github')
os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
!git clone  https://{GITHUB_TOKEN}@github.com/Kussil/Financial_Sentiment_LLM.git


Cloning into 'Financial_Sentiment_LLM'...
remote: Enumerating objects: 2973, done.
remote: Counting objects: 100% (539/539), done.
remote: Compressing objects: 100% (243/243), done.
remote: Total 2973 (delta 314), reused 504 (delta 292), pack-reused 2434
Receiving objects: 100% (2973/2973), 509.37 MiB | 13.59 MiB/s, done.
Resolving deltas: 100% (1946/1946), done.
Updating files: 100% (1212/1212), done.


# LLama code

In [12]:
# Define the template
TEMPLATE = """<s>Classify the following article into categories with sentiment (Positive, Neutral, Negative, N/A if not applicable or not mentioned) and provide the output in the specified dictionary format.
Example:
Article: ExxonMobil announced a significant increase in quarterly profits due to rising oil prices and increased production levels.
Output: {{"Finance": "Positive", 'Production': "Positive", "Reserves / Exploration / Acquisitions / Mergers / Divestments": 'Neutral', "Environment / Regulatory / Geopolitics": 'Neutral', "Alternative Energy / Lower Carbon": 'Neutral', "Oil Price / Natural Gas Price / Gasoline Price": "Positive"}}

Example:
Article: Chevron plans to invest heavily in renewable energy projects, aiming to reduce its carbon footprint over the next decade.
Output: {{'Finance': 'Neutral', 'Production': 'Neutral', "Reserves / Exploration / Acquisitions / Mergers / Divestments": 'Neutral', "Environment / Regulatory / Geopolitics": "Positive", "Alternative Energy / Lower Carbon": "Positive", "Oil Price / Natural Gas Price / Gasoline Price": 'Neutral'}}

Example:
Article: BP faced regulatory challenges in its latest drilling project, delaying operations and increasing costs.
Output: {{'Finance': 'Negative', "Production": 'Negative', "Reserves / Exploration / Acquisitions / Mergers / Divestments": 'Negative', "Environment / Regulatory / Geopolitics": 'Negative', "Alternative Energy / Lower Carbon": 'Neutral', "Oil Price / Natural Gas Price / Gasoline Price": 'Neutral'}}

Article: {article}

Output only the EXACT dictionary format:
{{"Finance": '[Sentiment]', "Production": '[Sentiment]', "Reserves / Exploration / Acquisitions / Mergers / Divestments": '[Sentiment]', "Environment / Regulatory / Geopolitics":: '[Sentiment]', "Alternative Energy / Lower Carbon": '[Sentiment]', "Oil Price / Natural Gas Price / Gasoline Price": '[Sentiment]'}}

Do not use any other format or additional information. Please provide the output in the specified format only.</s>"""

In [8]:
def process_article(article, model, tokenizer, generation_pipeline):
    """
    Processes an article using a language model to classify it into predefined categories with sentiment analysis.

    Args:
        article (str): The text of the article to be processed.
        model: The language model used for text generation and classification.
        tokenizer: Tokenizer used to encode the input text for the model.
        generation_pipeline: Pipeline or method for generating text outputs from the model.

    Returns:
        dict or str: A dictionary containing categorized sentiments ('Positive', 'Neutral', 'Negative', or 'N/A' for not applicable)
                    for predefined categories ('Finance', 'Production', 'Reserves / Exploration / Mergers', 'Environment / Regulatory',
                    'Alternative Energy', 'Oil/Gas Prices'), or a message indicating no valid JSON output found after 5 attempts.
    """

    for attempt in range(5):  # Attempt to find valid JSON 5 times
        # Create a prompt using the template
        prompt = TEMPLATE.format(article=article)

        # Encode the prompt and truncate to max length
        inputs = tokenizer(prompt, max_length=1024, truncation=True, return_tensors='pt')

        # Move inputs to the correct device
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        # Generate the classification using the LLaMA model
        result = model.generate(**inputs, max_new_tokens=200, num_beams=2, early_stopping=True)

        # Decode the result
        generated_text = tokenizer.decode(result[0], skip_special_tokens=True)

        # Split the response to separate the input prompt and the output
        split_response = generated_text.split("</s>", 1)
        if len(split_response) > 1:
            final_response = split_response[1].strip()
        else:
            final_response = generated_text.strip()

        # Extract the JSON part from the final response
        output_start = final_response.find("Output:")
        if output_start != -1:
            output_text = final_response[output_start:]
            json_output = re.search(r"\{.*\}", output_text)
            if json_output:
                try:
                    return json.loads(json_output.group(0).replace("'", '"'))
                except json.JSONDecodeError:
                    continue  # Continue to next attempt if JSON decoding fails
        else:
            continue  # Continue to next attempt if no "Output:" is found

    return "No valid JSON output found after 5 attempts."


# Model and Pipeline settings

In [9]:
# Load the LLaMA model and tokenizer
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    device_map="auto",
    quantization_config=quantization_config,
    use_auth_token=hf_token
)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", use_auth_token=hf_token)

# Create text generation pipeline
generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",
    max_length=8192,  # increase to max token size supported by the model
    truncation=True,
    do_sample=True,
    top_k=5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:778: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Data processing

In [10]:
# Path to the CSV file containing articles
#invest_df1 = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Investment_Research_Part1.csv')
#invest_df2 = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Investment_Research_Part2.csv')
proquest_df = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/ProQuest_Articles.csv')
#earnings_presentations = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Earnings_Presentations.csv')
#earnings_qa = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Earnings_QA.csv')
#sec_df = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/SEC_Filings.csv')

# Merge into single df
text_df = proquest_df
#text_df = pd.concat([invest_df1, invest_df2, proquest_df, sec_df, earnings_presentations, earnings_qa], ignore_index=True)
display(text_df.shape)
display(text_df.head())
display(text_df.tail())

(3708, 7)

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,ProQuest,PQ-2703147472,CVX,2022-08-17,Oil Giants Must Face Climate-Liability Suits i...,Chevron Corp.\nSpending projections are slidin...,https://www.proquest.com/newspapers/chevron-ge...
1,ProQuest,PQ-2702197510,CVX,2022-08-15,Warren Buffett's Berkshire Hathaway Keeps Spen...,Chevron Corp. said it would cut its annual cap...,https://www.proquest.com/newspapers/chevron-sl...
2,ProQuest,PQ-2697049245,CVX,2022-08-02,"Investors Put Forward More Proposals, Dialing ...",Chevron Corp. said it would cut its annual cap...,https://www.proquest.com/newspapers/chevron-sl...
3,ProQuest,PQ-2696371267,CVX,2022-07-30,Soaring Fuel Prices Catapult Oil Giants to Rec...,SENATORS aired misgivings on Tuesday over pote...,https://www.proquest.com/newspapers/senators-a...
4,ProQuest,PQ-2695835797,CVX,2022-07-29,"Exxon, Chevron, Shell Report Record Profits on...",LONDON—Royal Dutch Shell PLC said it would sta...,https://www.proquest.com/newspapers/shell-trie...


,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
3703,ProQuest,PQ-2238653651,PSX,2019-06-12,"As the Price of Gas Tumbles, Even California I...",Gasoline prices have been falling fast in rece...,https://www.proquest.com/newspapers/as-price-g...
3704,ProQuest,PQ-2239478009,PSX,2019-06-10,"Phillips 66, Plains plan pipeline system",CUSHING Phillips 66 and Plains All American Pi...,https://www.proquest.com/newspapers/phillips-6...
3705,ProQuest,PQ-2238860015,PSX,2019-06-07,Autonomous robot inspects in-service diesel st...,"HOUSTON Phillips 66, Square Robot Inc. and Ver...",https://www.proquest.com/newspapers/autonomous...
3706,ProQuest,PQ-2233473767,PSX,2019-06-03,United States/Mexico: Report: Trump's Tariff o...,US President Donald Trump's threats to tax Mex...,https://www.proquest.com/newspapers/united-sta...
3707,ProQuest,PQ-2232566793,PSX,2019-05-31,Trump's Mexico Tariffs Could Hit Americans at ...,President Trump's\nproposed tariffs on Mexico\...,https://www.proquest.com/newspapers/trumps-mex...


In [ ]:
# Path to the intermediate output file
intermediate_output_file = "/content/drive/MyDrive/Capstone folder/Sentiment_Framework/LLAMA_results/to_check_output_results_intermediate.json"
final_output_file = "/content/drive/MyDrive/Capstone folder/Sentiment_Framework/LLAMA_results/to_check_output_results_final.json"

# Name of the column containing the article text
article_column = 'Article Text'

# Output dictionary to store results
output_dict = {}

# Check if the intermediate output file exists and load it
if os.path.exists(intermediate_output_file):
    with open(intermediate_output_file, "r") as file:
        output_dict = json.load(file)
    # Find the ID of the last processed article in the output_dict
    last_processed_id = list(output_dict.keys())[-1]
    # Find the index of the last processed article in the DataFrame
    start_index = text_df[text_df['Unique_ID'] == last_processed_id].index[0] + 1
else:
    # If intermediate output file doesn't exist, start processing from the beginning
    start_index = 0

# Process each article in the DataFrame starting from the last saved state
for index, row in tqdm(text_df.iloc[start_index:].iterrows(), total=len(text_df) - start_index):
    unique_id = row['Unique_ID']
    article = row[article_column]

    output = process_article(article, model, tokenizer, generation_pipeline)
    output_dict[unique_id] = output
    print(f"Processed article {unique_id}")

    # Save the output dictionary to a JSON file every 1 iterations
    if (index + 1) % 1 == 0:
        with open(intermediate_output_file, "w") as file:
            json.dump(output_dict, file, indent=4)
        print(f"Intermediate results saved to {intermediate_output_file}")

# Save the final output dictionary to a JSON file
with open(final_output_file, "w") as file:
    json.dump(output_dict, file, indent=4)

print(f"Final results saved to {final_output_file}")

  0%|          | 0/3698 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  0%|          | 1/3698 [02:42<166:37:26, 162.25s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Processed article PQ-2665454855
Intermediate results saved to /content/drive/MyDrive/Capstone folder/Sentiment_Framework/LLAMA_results/to_check_output_results_intermediate.json


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  0%|          | 2/3698 [05:24<166:35:14, 162.26s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Processed article PQ-2665180174
Intermediate results saved to /content/drive/MyDrive/Capstone folder/Sentiment_Framework/LLAMA_results/to_check_output_results_intermediate.json


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  0%|          | 3/3698 [08:06<166:32:22, 162.26s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

  0%|          | 12/3698 [17:57<55:00:59, 53.73s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Processed article PQ-2641311895
Intermediate results saved to /content/drive/MyDrive/Capstone folder/Sentiment_Framework/LLAMA_results/to_check_output_results_intermediate.json


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  0%|          | 13/3698 [20:39<88:36:56, 86.57s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Processed article PQ-2641007493
Intermediate results saved to /content/drive/MyDrive/Capstone folder/Sentiment_Framework/LLAMA_results/to_check_output_results_intermediate.json


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
